This notebook follows from the file `hetero_2_3_analyse_the_annual_ts.py`.
# Read the data

In [1]:
import h5py
import numpy as np

filename = r"C:\SUSTech\datasets_of_graduation_project\hetero_outputs\overall_slopes.h5"

with h5py.File(filename, 'r') as f:
    ALPHAS = f['alphas'][:]
    LATS = f['latitudes'][:]
    LONS = f['longitudes'][:]
    overall_slopes = f['overall_slopes'][:]
    
print("The shape of slopes is: (len(ALPHAS), len(LATS), len(LONS)) = ({}, {}, {})".format(len(ALPHAS), len(LATS), len(LONS)))

The shape of slopes is: (len(ALPHAS), len(LATS), len(LONS)) = (5, 60, 144)


# Visualizations

In [2]:
overall_slopes[0]

array([[-0.0019521 , -0.0019521 , -0.0019521 , ..., -0.0019521 ,
        -0.0019521 , -0.0019521 ],
       [-0.00214466, -0.0020929 , -0.00205344, ..., -0.0022859 ,
        -0.00223436, -0.00214973],
       [-0.00226336, -0.00214284, -0.00212462, ..., -0.00205674,
        -0.00215474, -0.00225785],
       ...,
       [ 0.01950992,  0.01846384,  0.01704028, ...,  0.02045832,
         0.02283233,  0.01954606],
       [ 0.01884934,  0.01751255,  0.01802557, ...,  0.02373199,
         0.02165574,  0.018789  ],
       [ 0.01815735,  0.01608545,  0.0165155 , ...,  0.02377413,
         0.02064434,  0.01833357]], dtype=float32)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import matplotlib.colors as mcolors  # 颜色归一化

# 生成规则网格数据（LATS, LONS 笛卡尔积）
lons, lats = np.meshgrid(LONS, LATS)  # 生成规则网格

for alpha_idx, alpha in enumerate(ALPHAS):

    overall_slopes_alpha = overall_slopes[alpha_idx]  
    slope_min, slope_max = np.min(overall_slopes_alpha.flatten()), np.max(overall_slopes_alpha.flatten())
    print("The min and max of slopes are: ", slope_min, slope_max)

    # 创建北极投影地图
    fig, ax = plt.subplots(figsize=(8, 8))
    m = Basemap(projection="npstere", boundinglat=60, lon_0=0, resolution="l", ax=ax)

    # 转换网格点到投影坐标（Basemap 需要 x, y 坐标）
    x, y = m(lons, lats)

    # slope_abs_max = max(abs(slope_min), abs(slope_max))
    # norm = mcolors.Normalize(vmin=-slope_abs_max, vmax=slope_abs_max)  # 颜色归一化
    slope_abs_min = min(abs(slope_min), abs(slope_max))
    norm = mcolors.Normalize(vmin=-slope_abs_min, vmax=slope_abs_min)  # 颜色归一化

    # 绘制热力图
    c = m.pcolormesh(x, y, overall_slopes_alpha, cmap="coolwarm", norm=norm, shading="auto")

    # 绘制地图元素
    m.drawcoastlines()
    m.drawcountries()
    m.drawparallels(np.arange(60, 91, 10), labels=[True, True, False, False])
    m.drawmeridians(np.arange(-180, 181, 30), labels=[False, False, False, True])

    # 添加颜色条，并设置范围
    cb = plt.colorbar(c)
    # cb.set_ticks([-slope_abs_max, 0, slope_abs_max])
    cb.set_ticks([-slope_abs_min, 0, slope_abs_min])

    # 标题
    plt.title("Yearly increase of " + str(alpha) + "th percentile of 2m temp (°C/yr)")

    fig.savefig(r"C:\SUSTech\datasets_of_graduation_project\hetero_outputs\fig_outputs\sa\heatmap_of_2mtemp_annual" + str(alpha) + ".png", dpi=300)
    plt.close(fig)  # 关闭图形以释放内存